In [2]:
import sys
import numpy as np
import random
import pandas as pd
import pickle
import time
import alt_min

In [ ]:
U = np.random.standard_normal((2000, 40))
V = np.random.standard_normal((40, 2000))
L = np.matmul(U, V)

In [ ]:
p = 0.1
eta = 0.1
c = 100
mask_eta = np.random.rand(L.shape[0], L.shape[1])
S = np.random.uniform(-c, c, L.shape)
S[mask_eta < 1 - eta] = 0

In [ ]:
M = L + S
mask_p = np.random.rand(M.shape[0], M.shape[1])
mask_p[mask_p < 1 - p] = 0
mask_p[mask_p != 0] = 1
M_obs = np.multiply(M, mask_p)

In [ ]:
user_col = []
item_col = []
rating_col = []
for i in range(M_obs.shape[0]):
  for j in range(M_obs.shape[1]):
    rating = M_obs[i][j]
    if rating != 0:
      user_col.append(i)
      item_col.append(j)
      rating_col.append(rating)
train = pd.DataFrame.from_dict({"user": user_col, "item": item_col, "rating": rating_col})

In [ ]:
user_col = []
item_col = []
rating_col = []
for i in range(L.shape[0]):
  for j in range(L.shape[1]):
    rating = L[i][j]
    user_col.append(i)
    item_col.append(j)
    rating_col.append(rating)
test = pd.DataFrame.from_dict({"user": user_col, "item": item_col, "rating": rating_col})

In [ ]:
model = alt_min.LeastSquares(train, test, 2000, reg=0)
model.alt_min()

In [ ]:
train.loc[0][0]

In [ ]:
train["user"]

In [63]:
n = 10000
d = 30
eta = 0.1
X = np.random.standard_normal((n, d))
corrupt_idx = np.random.choice(n, (int)(eta * n), replace=False)
X[corrupt_idx] = np.random.uniform(0, 1000, (len(corrupt_idx), d))
est = alt_min.agnosticMeanGeneral(X, eta)
index_array = np.argsort(np.linalg.norm(X, axis=1))
robust_mean = np.mean(X[index_array][:(int)((1 - eta) * X.shape[0])], axis=0)
print("mean norm: {}".format(np.linalg.norm(np.mean(X, axis=0))))
print("median norm: {}".format(np.linalg.norm(np.median(X, axis=0))))
print("est norm: {}".format(np.linalg.norm(est)))
print("robust norm: {}".format(np.linalg.norm(robust_mean)))

mean norm: 272.2268973453166
median norm: 0.7616725546163047
est norm: 0.10511281170789671
robust norm: 0.055676777861700805


In [58]:
robust_mean.shape

(30,)

In [44]:
index_array

array([ 4, 15,  1,  6, 10, 13,  8,  5,  3, 17,  2, 12,  9,  0, 11, 18, 16,
        7, 14, 19])

In [47]:
X

array([[-1.19282474],
       [-0.2358787 ],
       [ 0.83768884],
       [-0.67290617],
       [ 0.2156672 ],
       [ 0.62902659],
       [-0.4714253 ],
       [ 1.61692698],
       [ 0.59423301],
       [-0.9857446 ],
       [-0.47262554],
       [-1.22230026],
       [-0.89191167],
       [ 0.57553509],
       [49.06853728],
       [ 0.22881141],
       [ 1.5529134 ],
       [-0.80332523],
       [-1.32488321],
       [99.00314456]])

In [46]:
X[index_array][:(int)((1 - eta) * X.shape[0])]

array([[ 0.2156672 ],
       [ 0.22881141],
       [-0.2358787 ],
       [-0.4714253 ],
       [-0.47262554],
       [ 0.57553509],
       [ 0.59423301],
       [ 0.62902659],
       [-0.67290617],
       [-0.80332523],
       [ 0.83768884],
       [-0.89191167],
       [-0.9857446 ],
       [-1.19282474],
       [-1.22230026],
       [-1.32488321],
       [ 1.5529134 ],
       [ 1.61692698]])

In [23]:
np.linalg.norm(robust_mean)

11.18050520654395

In [49]:
robust_mean

array([-0.11239016])

In [45]:
np.linalg.norm(X, axis=1).astype(int)

array([ 1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0, 49,  0,  1,
        0,  1, 99])